# Capstone Project - Battle Of The London Boroughs
## Where do I move next?

## Intorduction
This report is the final submission to the IBM Data Science Professional Certification Coursera course. Its aim is to demonstrate how today's data scientists can utilise the free APIs and open-source machine learning libraries to provide meaningful insights buisnes insights.

## The problem to solve
An estate agency company has approached me. They wish to provide their customers with peace of mind that the area they are moving to is precisely what they are looking for. Using the machine learning techniques I have learned throughout this course, I will analyse a number of London Boroughs to identify the similarities and differences between potential places to live. The agency will then be able to tell their customers if the area they are moving to is anything like where they are coming from or whether it is an entirely different vibe. 


## The Data
This project will utilise a number of data sources:
<ul>
    <li> The arcgis API for python </li>
    <li> The Foursquare API </li>
    <li> Postcodes.io API </li>
</ul>


### Geocoder Library and ArcGIS
The ArcGIS and Geocoder libraries can be used togeher to retrieve geographical data for specific addresses. <br>
Link: https://developers.arcgis.com

In [3]:
import geocoder

In [16]:
# National Physics Laboratory - Teddington, London
npl_geoData = geocoder.arcgis('TW11 0LW')
npl_geoData.geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'address': 'TW11 0LW',
    'bbox': [-0.34182997606174104,
     51.42260001403074,
     -0.33982997606174103,
     51.42460001403074],
    'confidence': 9,
    'lat': 51.42360001403074,
    'lng': -0.34082997606174104,
    'ok': True,
    'quality': 'PostalExt',
    'raw': {'name': 'TW11 0LW',
     'extent': {'xmin': -0.34182997606174104,
      'ymin': 51.42260001403074,
      'xmax': -0.33982997606174103,
      'ymax': 51.42460001403074},
     'feature': {'geometry': {'x': -0.34082997606174104,
       'y': 51.42360001403074},
      'attributes': {'Score': 100, 'Addr_Type': 'PostalExt'}}},
    'score': 100,
    'status': 'OK'},
   'bbox': [-0.34182997606174104,
    51.42260001403074,
    -0.33982997606174103,
    51.42460001403074],
   'geometry': {'type': 'Point',
    'coordinates': [-0.34082997606174104, 51.42360001403074]}}]}

### Forsquare API
The Forsquare API can be utilised to return venues and locations within a given radious of a specific point. <br>
Link: https://foursquare.com

In [21]:
import requests
from dotenv import dotenv_values

config = dotenv_values('.env')

four_id = config['CLIENT_ID']
four_sc = config['CLIENT_SECRET']

In [22]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    four_id, 
    four_sc, 
    20120609,
    npl_geoData.lat, 
    npl_geoData.lng,
    # Search radious 100m
    100, 
    # Top 100 results
    100)

results = requests.get(url).json()

In [34]:
# Here we see that there aren't many (1) locations of interest within 100m of the National Physics Laboratory 
results['response']

{'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
 'headerLocation': 'Teddington',
 'headerFullLocation': 'Teddington, London',
 'headerLocationGranularity': 'neighborhood',
 'totalResults': 1,
 'suggestedBounds': {'ne': {'lat': 51.42450001493074,
   'lng': -0.3393893385939807},
  'sw': {'lat': 51.422700013130736, 'lng': -0.3422706135295014}},
 'groups': [{'type': 'Recommended Places',
   'name': 'recommended',
   'items': [{'reasons': {'count': 0,
      'items': [{'summary': 'This spot is popular',
        'type': 'general',
        'reasonName': 'globalInteractionReason'}]},
     'venue': {'id': '59da11ce9b04736749c52a90',
      'name': 'Rich Coffee Company',
      'contact': {},
      'location': {'lat': 51.42363841432043,
       'lng': -0.3407308309310047,
       'labeledLatLngs': [{'label': 'display',
         'lat': 51.42363841432043,
         'lng': -0.3407308309310047}],
       'distance':

### Postcode.io API
The postcode API allows users to send request based on the latitude and longitude or postcode of a location and retrieve information for that area. <br>
Link: https://postcodes.io

In [47]:
url_codes = 'https://api.postcodes.io/postcodes?lon={}&lat={}'.format(
    npl_geoData.lng, 
    npl_geoData.lat
)

codes = requests.get(url_codes).json()
codes['result'][0]

{'postcode': 'TW11 0LW',
 'quality': 1,
 'eastings': 515458,
 'northings': 170734,
 'country': 'England',
 'nhs_ha': 'London',
 'longitude': -0.340797,
 'latitude': 51.423865,
 'european_electoral_region': 'London',
 'primary_care_trust': 'Richmond and Twickenham',
 'region': 'London',
 'lsoa': 'Richmond upon Thames 021C',
 'msoa': 'Richmond upon Thames 021',
 'incode': '0LW',
 'outcode': 'TW11',
 'parliamentary_constituency': 'Twickenham',
 'admin_district': 'Richmond upon Thames',
 'parish': 'Richmond upon Thames, unparished area',
 'admin_county': None,
 'admin_ward': 'Teddington',
 'ced': None,
 'ccg': 'NHS South West London',
 'nuts': 'Hounslow and Richmond upon Thames',
 'codes': {'admin_district': 'E09000027',
  'admin_county': 'E99999999',
  'admin_ward': 'E05000530',
  'parish': 'E43000217',
  'parliamentary_constituency': 'E14001005',
  'ccg': 'E38000245',
  'ccg_id': '36L',
  'ced': 'E99999999',
  'nuts': 'UKI75',
  'lsoa': 'E01003888',
  'msoa': 'E02000804',
  'lau2': 'E050